Thanks https://www.kaggle.com/cdeotte

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
import matplotlib.pyplot as plt
path = '/kaggle/input/h-and-m-personalized-fashion-recommendations/'
impath = '/kaggle/input/h-and-m-personalized-fashion-recommendations/images/0'
from tqdm import tqdm

In [ ]:
relation = pd.read_csv(f'{path}transactions_train.csv')
relation = relation[['customer_id','article_id','t_dat','sales_channel_id']]
print(relation.shape)

In [ ]:
channel = relation.groupby('customer_id')['sales_channel_id'].mean().apply(lambda x: 1 if(x<1.5) else 2)
channel.head()

# Clustering Users into 310 Groups

In [ ]:
users = pd.read_csv(f'{path}customers.csv')
users = users[['customer_id', 'FN', 'Active', 'club_member_status',
       'fashion_news_frequency','age']]
users = users.merge(channel,on='customer_id',how='left')
print(users.shape)

In [ ]:
bin_list = [0, 10,15,18,22,25,30,35,40,45,50,55,60,65,100]
users['age_bins'] = pd.cut(users['age'], bin_list)
users['FN'] = users['FN'].replace(np.nan,0)
users['Active'] = users['Active'].replace(np.nan,0)
users.club_member_status = users.club_member_status.replace(['PRE-CREATE','LEFT CLUB',np.nan],'PASSIVE')
users.fashion_news_frequency = users.fashion_news_frequency.replace(['None',np.nan],'NONE')
users.fashion_news_frequency = users.fashion_news_frequency.replace('Monthly','Regularly')
users.drop(columns='age',inplace=True)
users['age_bins'] = users['age_bins'].replace(np.nan,'popular')
users['sales_channel_id'] = users['sales_channel_id'].replace(np.nan,3)
for i in ['club_member_status','fashion_news_frequency','age_bins'] :
    users[i] = le.fit_transform(users[i])
users['cluster'] = users[['FN','Active','club_member_status','fashion_news_frequency','age_bins','sales_channel_id']].apply(tuple,axis=1)
users['cluster']=users['cluster'].astype(str)
users['cluster'] = le.fit_transform(users['cluster'])
users.drop(columns=['FN','Active','club_member_status','fashion_news_frequency','age_bins','sales_channel_id'],inplace=True)
users

# Clustering Items into 3398 Groups

In [ ]:
items = pd.read_csv(f'{path}articles.csv')
items = items[['article_id','product_type_name', 'perceived_colour_master_name', 'index_group_name']]
print(items.shape)

In [ ]:
for i in items.columns[1:] :
    items[i] = le.fit_transform(items[i])
items['clusteri'] = items[['product_type_name','perceived_colour_master_name','index_group_name']].apply(tuple,axis=1)
items['clusteri']= items['clusteri'].astype(str)
items['clusteri'] = le.fit_transform(items['clusteri'])
#items.drop(columns=['product_type_name','perceived_colour_master_name','index_group_name'],inplace=True)
items

# Merging to Relation

In [ ]:
relation = relation.merge(users,how='left',on='customer_id')
relation = relation.merge(items.drop(columns=['product_type_name','perceived_colour_master_name','index_group_name']),how='left',on='article_id')
relation

# Popular Items - TimeWise

In [ ]:
#newusers = [706016001,751471001,924243001,372860001,915529003,918522001,
#            610776002,923758001,918292001, 915526001,730683050, 448509014]
newusers = [909370001, 924243001, 918522001, 865799006, 751471001, 915529003,
            918292001, 915526001, 706016001, 751471043, 372860002, 372860001]

### Each clusters 6 popular item

In [ ]:
week = relation[(relation['t_dat'] > '2020-09-08')]
month = relation[(relation['t_dat'] > '2020-08-22')]
quarter = relation[(relation['t_dat'] > '2020-06-22')]

usergroup = dict()
for i in tqdm (range(311), desc="Loading..."):
    top6all = list(relation[(relation['cluster']==i)].groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(6).index.astype(int))
    top6week = list(week[(week['cluster']==i)].groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(6).index.astype(int))
    top6month = list(month[(month['cluster']==i)].groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(6).index.astype(int))
    top6quarter = list(quarter[(quarter['cluster']==i)].groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(6).index.astype(int))
    
    cnt = 8
    ls = []
    for j in top6week:
        if(j not in ls):
            ls.append(j)
            cnt=cnt-1
            if(cnt==6):
                break
    for j in top6month:
        if(j not in ls):
            ls.append(j)
            cnt=cnt-1
            if(cnt==4):
                break
    for j in top6quarter:
        if(j not in ls):
            ls.append(j)
            cnt=cnt-1
            if(cnt==2):
                break
    for j in top6all:
        if(j not in ls):
            ls.append(j)
            cnt=cnt-1
            if(cnt==0):
                break
    usergroup[i]=ls


itemgroup = dict()
for i in tqdm (range(3398), desc="Loading..."):
    top6all = list(relation[(relation['clusteri']==i)].groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(6).index.astype(int))
    top6week = list(week[(week['clusteri']==i)].groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(6).index.astype(int))
    top6month = list(month[(month['clusteri']==i)].groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(6).index.astype(int))
    top6quarter = list(quarter[(quarter['clusteri']==i)].groupby('article_id')['customer_id'].nunique().sort_values(ascending=False).head(6).index.astype(int))

    cnt = 8
    ls = []
    for j in top6week:
        if(j not in ls):
            ls.append(j)
            cnt=cnt-1
            if(cnt==6):
                break
    for j in top6month:
        if(j not in ls):
            ls.append(j)
            cnt=cnt-1
            if(cnt==4):
                break
    for j in top6quarter:
        if(j not in ls):
            ls.append(j)
            cnt=cnt-1
            if(cnt==2):
                break
    for j in top6all:
        if(j not in ls):
            ls.append(j)
            cnt=cnt-1
            if(cnt==0):
                break
    itemgroup[i]=ls

In [ ]:
#usergroup = np.load('/kaggle/input/hm-data/usergroup.npy',allow_pickle=True).item()
#itemgroup = np.load('/kaggle/input/hm-data/itemgroup.npy',allow_pickle=True).item()

# Items Purchased Together

In [ ]:
pairs = np.load('/kaggle/input/hm-data/pairs_cudf.npy',allow_pickle=True).item()

# User's Most Bought

In [ ]:
tmp = relation.groupby(['customer_id','article_id'])['t_dat'].agg('count').reset_index()
tmp.columns = ['customer_id','article_id','ct']
tmp = relation.merge(tmp,on=['customer_id','article_id'],how='left')
tmp = tmp.sort_values(['ct','t_dat'],ascending=False)
tmp = tmp.drop_duplicates('customer_id',keep='first')
tmp = tmp[['customer_id', 'article_id', 'clusteri']]
tmp.columns = ['customer_id', 'farticle_id', 'fclusteri']
tmp.head()

# User Latest Purchase

In [ ]:
relation = relation.drop_duplicates(subset='customer_id',keep='last')
relation = relation.reset_index(drop=True)
relation.head()

# Merge

In [ ]:
relation = relation.merge(tmp,on='customer_id')
relation.head()

# Recommendations

### Picking 12 Items
#### 1. Last Purchase and it's pair - 2
#### 2. Frequent Purchase and it's pair - 2  
#### 3. User Group - 4  
#### 4. Item Group Last - 2 
#### 5. Item Group Frequent - 2

#### Incase less than 12 then from New Users List

In [ ]:
rec = dict()
for i in tqdm (range(relation.shape[0]), desc="Loading..."):
    flag=0
    cnt = 12
    # Last purchase
    ls = []
    lastpur= relation.iloc[i]['article_id']
    ls.append(lastpur)
    rec[i] = '0' +str(lastpur)
    cnt=11
    # It's pair
    try:
        if(lastpur!=pairs[relation.iloc[i]['article_id']]):
            ls.append(pairs[relation.iloc[i]['article_id']])
            rec[i] += ' 0'+str(pairs[relation.iloc[i]['article_id']])
            cnt-=1
        else:
            flag+=1 
    except:
        flag+=1
    # Most Frequent
    lastpur = relation.iloc[i]['farticle_id']
    if(relation.iloc[i]['farticle_id'] not in ls):
        rec[i]+= ' 0'+str(j)
        ls.append(lastpur)
        cnt-=1
    else:
        flag+=1 
    # It's pair
    try:
        if((lastpur!=pairs[relation.iloc[i]['farticle_id']]) & (lastpur not in ls)):
            ls.append(pairs[relation.iloc[i]['farticle_id']])
            rec[i] += ' 0'+str(pairs[relation.iloc[i]['farticle_id']])
            cnt-=1
        else:
            flag+=1 
    except:
        flag+=1
    # 4 items from the same user group 
    for j in usergroup[relation.iloc[i]['cluster']]:
        if(j not in ls):
            rec[i]+= ' 0'+str(j)
            ls.append(j)
            cnt-=1
        else:
            flag+=1
        if(cnt==4+flag):
            break
    for j in itemgroup[relation.iloc[i]['clusteri']]:
        if(j not in ls):
            rec[i]+= ' 0'+str(j)
            ls.append(j)
            cnt-=1
        else:
            flag+=1
        if(cnt==2+flag):
            break  
    for j in itemgroup[relation.iloc[i]['fclusteri']]:
        if(j not in ls):
            rec[i]+= ' 0'+str(j)
            ls.append(j)
            cnt-=1
        else:
            flag+=1
        if(cnt==flag):
            break  
    if(flag!=0):
        for j in newusers:
            if(j not in ls):
                rec[i]+= ' 0'+str(j)
                ls.append(j)
                flag-=1
            if(flag==0):
                break  

In [ ]:
relation = relation['customer_id']
relation = pd.concat([relation,pd.Series(rec)],axis=1,ignore_index=True)
relation.columns =['customer_id','prediction']
final = pd.merge(users,relation, on='customer_id',how='left').drop(columns='cluster')

In [ ]:
newusers = '0909370001 0924243001 0918522001 0865799006 0751471001 0915529003 0918292001 0915526001 0706016001 0751471043 0372860002 0372860001'

In [ ]:
final = final.replace(np.nan,newusers)

In [ ]:
final.to_csv('submission.csv',index=False)